In [1]:
import sys
import os
sys.path.insert(0, '..')

In [2]:
import time


In [3]:
import logging
import pandas as pd
import numpy as np
from pathlib import Path
from functools import reduce


from lmkgroup_ds_utils.db.connector import DB
from lmkgroup_ds_utils.constants import Company
from customer_churn.paths import SQL_DIR

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
company_id = Company.GL
db = DB(
    local=True,
    db_name="analytics_db"
)

postgres_db = DB(
    local=True,
    db_name="postgres_db",
)
model_training = True
start_date_var = "2024-02-29"
end_date_var =  "2024-03-07"
snapshot_dates = pd.date_range(start_date_var, end_date_var)

INFO:lmkgroup_ds_utils.db.connector:Connecting to database analytics_db using conn_str Driver={ODBC Driver 18 for SQL Server};Server=tcp:gg-analytics-rep.database.windows.net,1433;Database=AnalyticsDB;Uid=thomas;Pwd=3$40eFvZo6TdsbNo;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;
INFO:lmkgroup_ds_utils.db.connector:Connecting to database postgres_db using conn_str DRIVER=org.postgresql.Driver;DATABASE=staging;UID=thomas@gg-analytics-staging;SERVER=gg-analytics-staging.postgres.database.azure.com;PORT=5432;PWD=gfjoGdp3RvoMdrKa


In [5]:
from customer_churn.dataset.bisnode import Bisnode
from customer_churn.dataset.complaints import Complaints
from customer_churn.dataset.crm_segments import CRMSegments
from customer_churn.dataset.customers import Customers
from customer_churn.dataset.events import Events
from customer_churn.dataset.orders import Orders

In [6]:
### Bisnode data
bisnode = Bisnode(company_id, db, model_training)
bisnode.load()
bisnode.df

INFO:customer_churn.dataset.bisnode:Get bisnode data from database...
INFO:lmkgroup_ds_utils.db.connector:Running query on analytics_db
/Users/thomas.sve/LMK/sous-chef/packages/lmkgroup-ds-utils/lmkgroup_ds_utils/db/connector.py:122: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


,agreement_id,impulsiveness,created_at,cultural_class,perceived_purchasing_power,consumption,children_probability,financial_class,life_stage,type_of_housing,confidence_level
0,60343,73.0,2020-07-06 11:08:14.816666,_________,______,88.0,95.0,______,___________________,________,_____
3,60347,68.0,2020-07-06 11:08:14.816666,_________,______,99.0,95.0,______,___________________,___________,_____
4,60353,66.0,2020-07-06 11:08:14.816666,_________,_________,88.0,70.0,______,_________________________,________,_____
5,60355,76.0,2020-07-06 11:08:14.816666,_________,_________,80.0,59.0,______,___________________,___________,_____
6,60362,42.0,2020-07-06 11:08:14.816666,_________,______,86.0,45.0,______,_________________________,___________,_____
...,...,...,...,...,...,...,...,...,...,...,...
2607936,1349827,86.0,2024-03-11 22:30:06.480000,_________,_________,89.0,71.0,______,_________________________,___________,_____
2607937,1349829,86.0,2024-03-11 23:00:08.700000,_________,_________,95.0,76.0,_________,_________________________,___________,_____
2607939,1349834,49.0,2024-03-12 07:30:07.086666,_________,_________,70.0,16.0,______,_________________,________,_____
2607940,1349838,87.0,2024-03-12 08:30:07.763333,______,_________,80.0,95.0,______,___________________,___________,_____


In [7]:
### Complaints data
complaints = Complaints(company_id, db, model_training)
complaints.load()
complaints.df

INFO:customer_churn.dataset.complaints:Get complaints data from database...
INFO:lmkgroup_ds_utils.db.connector:Running query on analytics_db
/Users/thomas.sve/LMK/sous-chef/packages/lmkgroup-ds-utils/lmkgroup_ds_utils/db/connector.py:122: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


,agreement_id,delivery_year,delivery_week,category,registration_date,delivery_date
0,60696,2021,1,logistics,2021-01-05 10:03:29.537,2021-01-10
1,60761,2021,1,packing,2021-01-05 15:05:14.683,2021-01-10
2,60858,2021,1,customer service,2021-01-04 18:06:54.867,2021-01-10
3,60858,2021,1,packing,2021-01-04 18:05:50.213,2021-01-10
4,61237,2021,1,procurement,2021-01-07 13:13:16.240,2021-01-10
...,...,...,...,...,...,...
119458,1063301,2022,46,customer service,2022-11-09 10:53:25.163,2022-11-20
119459,1290580,2023,24,customer service,2023-06-08 09:12:30.737,2023-06-18
119460,1236154,2022,39,customer service,2022-09-21 14:54:54.687,2022-10-02
119461,1085093,2021,40,customer service,2021-09-30 12:28:37.397,2021-10-10


In [8]:
### CRMSegment
crm_segment = CRMSegments(company_id, db, model_training)
crm_segment.load()
crm_segment.df

INFO:customer_churn.dataset.crm_segments:Get crm segment data from database...
INFO:lmkgroup_ds_utils.db.connector:Running query on analytics_db
/Users/thomas.sve/LMK/sous-chef/packages/lmkgroup-ds-utils/lmkgroup_ds_utils/db/connector.py:122: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


,agreement_id,current_delivery_year,current_delivery_week,planned_delivery
0,155794,2024,11,True
1,155816,2024,11,False
2,155825,2024,11,True
3,155832,2024,11,False
4,155844,2024,11,True
...,...,...,...,...
22635,1257411,2024,11,False
22636,1257543,2024,11,True
22637,1257559,2024,11,True
22638,1257567,2024,11,False


In [9]:
customers = Customers(company_id, db, model_training)
customers.load()
customers.df

INFO:customer_churn.dataset.customers:Get customers data from database...
INFO:lmkgroup_ds_utils.db.connector:Running query on analytics_db
/Users/thomas.sve/LMK/sous-chef/packages/lmkgroup-ds-utils/lmkgroup_ds_utils/db/connector.py:122: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
INFO:customer_churn.dataset.customers:Total customers with delete status (will be ignored): 0
INFO:customer_churn.dataset.customers:Total Customers: 60693


,agreement_id,agreement_status,agreement_start_date,agreement_start_year,agreement_start_week,agreement_first_delivery_year,agreement_first_delivery_week,agreement_creation_date,agreement_first_delivery_date,last_delivery_date,next_estimated_delivery_date
0,1247434,freezed,2022-10-16,2022,42,2022,42,2022-09-23 14:38:29.747,2022-10-16,2023-06-11,NaT
1,1059618,freezed,2021-04-25,2021,17,2021,17,2021-04-19 17:38:29.760,2021-04-25,2022-07-03,NaT
2,936786,freezed,2020-03-09,2020,11,2020,11,2020-02-27 13:48:46.653,2020-03-09,2023-10-09,NaT
3,1242443,active,2022-09-04,2022,36,2022,36,2022-08-28 20:01:22.273,2022-09-04,2024-03-10,2024-03-17
4,1346709,active,2024-03-03,2024,10,2024,10,2024-02-27 21:54:31.793,2024-03-04,2024-03-11,2024-03-17
...,...,...,...,...,...,...,...,...,...,...,...
60688,1340903,freezed,2024-02-04,2024,6,2024,6,2024-01-30 07:56:30.650,2024-02-04,2024-02-11,NaT
60689,1015970,freezed,2021-01-10,2021,2,2021,2,2020-12-28 13:05:23.987,2021-01-10,2022-03-13,NaT
60690,1074663,freezed,2021-08-16,2021,33,2021,33,2021-08-06 16:58:05.780,2021-08-16,2022-04-25,NaT
60691,1227877,freezed,2022-03-27,2022,13,2022,13,2022-03-12 11:15:23.663,2022-03-27,2022-05-22,NaT


In [10]:
db.conf_db

{'dbname': 'AnalyticsDB',
 'user': 'thomas',
 'passw': '3$40eFvZo6TdsbNo',
 'host': 'gg-analytics.database.windows.net',
 'port': 1433,
 'driver': 'ODBC Driver 18 for SQL Server',
 'connection_string': 'Driver={ODBC Driver 18 for SQL Server};Server=tcp:gg-analytics-rep.database.windows.net,1433;Database=AnalyticsDB;Uid=thomas;Pwd=3$40eFvZo6TdsbNo;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'}

In [11]:
postgres_db.conf_db

{'dbname': 'staging',
 'user': 'thomas@gg-analytics-staging',
 'passw': 'gfjoGdp3RvoMdrKa',
 'host': 'gg-analytics-staging.postgres.database.azure.com',
 'port': 5432,
 'driver': 'org.postgresql.Driver'}

In [12]:
events = Events(company_id, postgres_db, model_training)
start_time = time.time()
events.load()
print("--- %s seconds ---" % (time.time() - start_time))
events.df


INFO:customer_churn.dataset.events:Get events data from database...
INFO:lmkgroup_ds_utils.db.connector:Running query on postgres_db
/Users/thomas.sve/LMK/sous-chef/packages/lmkgroup-ds-utils/lmkgroup_ds_utils/db/connector.py:122: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


--- 3.902008056640625 seconds ---


,agreement_id,event_text,timestamp,account_event,normal_event,status_change
0,1216934,deviationOrdered,2023-01-03 17:22:33.419000+00:00,False,False,False
1,1211752,changeStatus_Activated,2023-01-03 15:58:52.235000+00:00,False,False,False
2,1020428,deviationOrdered,2023-01-03 16:35:02.185000+00:00,False,False,False
3,78793,changeStatus_Activated,2023-01-03 17:07:45.055000+00:00,False,False,False
4,1236839,deviationOrdered,2023-01-03 16:48:16.030000+00:00,False,False,False
...,...,...,...,...,...,...
995,1249471,changeStatus_Activated,2023-01-03 19:49:26.304000+00:00,False,False,False
996,1209281,deviationOrdered,2023-01-03 18:58:20.992000+00:00,False,False,False
997,1257330,deviationOrdered,2023-01-03 20:20:25.341000+00:00,False,False,False
998,1020130,deviationOrdered,2023-01-03 19:30:24.120000+00:00,False,False,False


In [13]:
orders = Orders(company_id, db, model_training)
orders.load()
orders.df

INFO:customer_churn.dataset.orders:Get orders data from database...
INFO:lmkgroup_ds_utils.db.connector:Running query on analytics_db
/Users/thomas.sve/LMK/sous-chef/packages/lmkgroup-ds-utils/lmkgroup_ds_utils/db/connector.py:122: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


,agreement_id,company_name,order_id,delivery_date,delivery_year,delivery_week,net_revenue_ex_vat,gross_revenue_ex_vat
0,164805,Godtlevert,4742400,2021-01-11,2021,2,651.304321,651.304321
1,1225693,Godtlevert,7598326,2022-11-07,2022,45,738.000000,738.000000
2,885786,Godtlevert,7841364,2023-01-23,2023,4,910.000000,910.000000
3,157423,Godtlevert,9031783,2024-01-29,2024,5,780.000000,780.000000
4,227830,Godtlevert,6542587,2022-01-30,2022,5,585.000000,881.000000
...,...,...,...,...,...,...,...,...
1362747,878297,Godtlevert,4745883,2021-01-10,2021,2,651.304321,651.304321
1362748,1218785,Godtlevert,6660854,2022-02-21,2022,8,279.000000,694.000000
1362749,1279326,Godtlevert,8249610,2023-05-22,2023,21,1187.000000,1221.000000
1362750,1240502,Godtlevert,8876851,2023-12-11,2023,50,813.000000,813.000000


## Get features for snapshot date

In [14]:
snapshot_date = snapshot_dates[0]

features = [
    "customer_since_weeks",
    "weeks_since_last_delivery",
    "number_of_forecast_orders",
    "number_of_total_orders",
    "snapshot_status",
] # and much more

label = "forecast_status"

In [20]:
snapshot_date

Timestamp('2024-02-29 00:00:00')

In [15]:
bisnode_features = bisnode.get_features_for_snapshot(snapshot_date)
bisnode_features.head()

,agreement_id,impulsiveness,created_at,cultural_class,perceived_purchasing_power,consumption,children_probability,financial_class,life_stage,type_of_housing,confidence_level
119592,60343,73.0,2024-02-02 02:28:36.083333,_________,______,88.0,95.0,______,___________________,________,_____
119626,60347,68.0,2023-10-23 02:01:05.770000,_________,______,99.0,95.0,______,___________________,___________,_____
119669,60353,66.0,2024-02-03 02:04:12.070000,_________,_________,88.0,70.0,______,_________________________,________,_____
119680,60355,76.0,2024-02-03 02:12:06.840000,_________,_________,80.0,59.0,______,___________________,___________,_____
119688,60362,42.0,2022-03-04 02:30:00.580000,_________,______,86.0,45.0,______,_________________________,___________,_____


In [16]:
complaints_features = complaints.get_features_for_snapshot(snapshot_date)
complaints_features.head()

,agreement_id,total_complaints,weeks_since_last_complaint,number_of_complaints_last_n_weeks,category
0,60343,1,79,0,logistics
1,60346,4,90,0,packing
2,60347,4,93,0,procurement
3,60364,1,122,0,customer
4,60366,8,2,1,menu


In [17]:
df = complaints.df.groupby("agreement_id").aggregate(
            total_complaints=pd.NamedAgg('agreement_id', 'count'),
            last_complaint=pd.NamedAgg('delivery_date', 'max'),
            category=pd.NamedAgg('category', 'last')
        )

In [18]:
df["weeks_since_last_complaint"] = (snapshot_date - df["last_complaint"])

In [19]:
df["weeks_since_last_complaint"].dt.days // 7

agreement_id
60343       79
60346       90
60347       93
60364      122
60366        2
          ... 
1347495     -3
1347566     -3
1347583     -3
1347692     -3
1347798     -3
Name: weeks_since_last_complaint, Length: 37431, dtype: int64